Assignment 1 - Building a Random Forest
This is a skeleton of a Random Forest classifier.



In [105]:
import csv
import math
from statistics import median, mode, mean
from collections import Counter
from enum import Enum
import numpy as np
import pandas as pd
import statistics

Some simple type definitions.

In [106]:
class AttrType(Enum):
    cat = 0  # categorical (qualitative) attribute
    num = 1  # numerical (quantitative) attribute
    target = 2  # target label

class NodeType(Enum):
    root = 0
    internal = 1
    leaf = 2

class SplitType(Enum):
    bin = 0  # binary split
    multi = 1  # multi-way split

Also, some basic classes to represent an attribute, a spltting procedure, and a node.

In [107]:
class Attribute(object):
    def __init__(self, label, type):
        assert type in AttrType
        self.label = label
        self.type = type
        self.stat = None  # holds mean for numerical and mode for categorical attributes


class Splitting(object):
    def __init__(self, attr, infogain, split_type, cond, splits):
        self.attr = attr  # attribute ID (index in ATTR)
        self.infogain = infogain  # information gain if splitting is done on this attribute
        self.split_type = split_type  # one of SplitType
        self.cond = cond  # splitting condition, i.e., values on outgoing edges
        self.splits = splits  # list of training records (IDs) for each slitting condition


class Node(object):
    def __init__(self, id, type, parent_id, children=None, edge_value=None, val=None, split_type=None, split_cond=None,
                 infogain=None):
        self.id = id  # ID (same as the index in DT.model list)
        self.type = type  # one of NodeType
        self.parent_id = parent_id  # ID of parent node (None if root)
        self.children = children  # list of IDs of child nodes
        self.edge_value = edge_value  # the value of the incoming edge (only if not root node)
        self.val = val  # if root or internal node: the attribute that is compared at that node; if leaf node: the target value
        self.split_type = split_type  # one of SplitType
        self.split_cond = split_cond  # splitting condition (median value for binary splits on numerical values; otherwise a list of categorical values (corresponding to child nodes))
        self.infogain = infogain

    def append_child(self, node_id):
        self.children.append(node_id)

In [108]:
df = pd.read_csv('housing_price_train.csv')
df['LotFrontage'] = df['LotFrontage'].fillna(df['LotFrontage'].median())
df.set_index('Id')
df.to_csv('housing_price_train1.csv', index=False)

In [109]:
ATTR = []
INFILE = "housing_price_train1.csv"

for col in df.columns:
    if col == "SalePrice":
        ATTR.append(Attribute(col, AttrType.target))
    elif col == "MSSubClass" or col == "OverallQual" or col == "OverallCond":
        ATTR.append(Attribute(col, AttrType.cat))
    elif df[col].unique().dtype == "int64" or df[col].unique().dtype == "float64":
        ATTR.append(Attribute(col, AttrType.num))        
    else:
        ATTR.append(Attribute(col, AttrType.cat))

IDX_TARGET = len(ATTR) - 1 

A class DT representing the decision tree classifier. It could represent with methods:

a given impurity measure;
the search for the best attribute to split with;
the addition of a node to the tree;
a convenient model printer;
the recursive call for obtaining a tree;
a builder and an applier.

In [110]:
class DT(object):
    def __init__(self):
        self.data = None  # training data set (loaded into memory)
        self.model = None  # decision tree model
        # self.default_target = 0.0  # default target class HVA ER DET FOR NOE?

    def __load_data(self):
        # with open(INFILE) as csvfile:
        #     self.data = []
        #     csvreader = csv.reader(csvfile, delimiter=',')
        #     next(csvreader)
        #     for row in csvreader:
        #         rec = []
        #         for i in range(len(ATTR)):
        #             val = row[i].strip()
        #             # convert numerical attributes
        #             if ATTR[i].type == AttrType.num:  # Note that this will break for "?" (missing attribute)
        #                 if val == "":
        #                     val = None
        #                 else:
        #                     val = float(val)
        #             rec.append(val)
        #         self.data.append(rec)
        #     # print(self.data)
        #         # self.data.append([element.strip() for element in row])  # strip spaces
        self.data = df.values.tolist()
                
    def __median(self, a):
        numList = []
        for x in range(a):
            numList.append(x)
        if len(numList) > 0:
            return statistics.median(numList)
        else:
            return 0

    # HVA BRUKES DETTE TIL?
    # def __mean_squared_error(self, records):
    #     """
    #     Calculates mean squared error for a selection of records.

    #     :param records: Data records (given by indices)
    #     """
    #     # TODO
    #     #print(records)
    #     return 0
        
    def __find_best_attr(self, attrs, records):
        """
        Finds the attribute with the largest gain.

        :param attrs: Set of attributes
        :param records: Training set (list of record ids)
        :return:
        """
        print("attrs:", attrs)
        print("records:", records)
        # mse_p = self.__mean_squared_error(records)  # parent's MSE HVORFOR TRENGER VI DEN?
        splittings = []  # holds the splitting information for each attribute

        for a in attrs:
            assert ATTR[a].type in AttrType
            splits = {}  # record IDs corresponding to each split
            # splitting condition depends on the attribute type
            if ATTR[a].type == AttrType.target:  # skip target attribute
                continue
            elif ATTR[a].type == AttrType.cat:  # categorical attribute
                # multi-way split on each possible value
                split_mode = SplitType.multi
                # each possible attr value corresponds to a split (indexed with categorical labels)
                # Note: it's important to consider attr values from the entire training set
                split_cond = set([self.data[idx][a] for idx in range(len(self.data))])
                
                # TODO collect training records for each split 
                # `splits[val]` holds a list of records for a given split,
                # where `val` is an element of `split_cond`
                print(type(splits))
            elif ATTR[a].type == AttrType.num:  # numerical attribute => binary split on median value
                split_mode = SplitType.bin
                split_cond = self.__median(a)  # (i.e., if less or equal than this value)
                # TODO collect training records for each split (in `splits`)

            # TODO compute gain for attribute a
            infogain = 0

            splitting = Splitting(a, infogain, split_mode, split_cond, splits)
            splittings.append(splitting)

        # find best splitting
        best_splitting = sorted(splittings, key=lambda x: x.infogain, reverse=True)[0]
        return best_splitting

    def __add_node(self, parent_id, node_type=NodeType.internal, edge_value=None, val=None, split_type=None,
                   split_cond=None):
        """
        Adds a node to the decision tree.

        :param parent_id:
        :param node_type:
        :param edge_value:
        :param val:
        :param split_type:
        :param split_cond:
        :return:
        """
        node_id = len(self.model)  # id of the newly assigned node
        if not self.model:  # the tree is empty
            node_type = NodeType.root

        node = Node(node_id, node_type, parent_id, children=[], edge_value=edge_value, val=val, split_type=split_type,
                    split_cond=split_cond)
        self.model.append(node)

        # also add it as a child of the parent node
        if parent_id is not None:
            self.model[parent_id].append_child(node_id)

        return node_id
    def __id3(self, attrs, records, parent_id=None, value=None):
        """
        Function ID3 that returns a decision tree.

        :param attrs: Set of attributes
        :param records: Training set (list of record ids)
        :param parent_id: ID of parent node
        :param value: Value corresponding to the parent attribute, i.e., label of the edge on which we arrived to this node
        :return:
        """
        # empty training set or empty set of attributes => create leaf node with default class
        if not records or not attrs:
            self.__add_node(parent_id, node_type=NodeType.leaf, edge_value=value, val=self.default_class)
            return

        # if all records have the same target value => create leaf node with that target value
        same = all(self.data[idx][IDX_TARGET] == self.data[records[0]][IDX_TARGET] for idx in records)
        if same:
            target = self.data[records[0]][IDX_TARGET]
            self.__add_node(parent_id, node_type=NodeType.leaf, edge_value=value, val=target)
            return

        # find the attribute with the largest gain
        splitting = self.__find_best_attr(attrs, records)
        # add node
        node_id = self.__add_node(parent_id, edge_value=value, val=splitting.attr, split_type=splitting.split_type,
                                  split_cond=splitting.cond)
        # TODO call tree construction recursively for each split

    def print_model(self, node_id=0, level=0):
        node = self.model[node_id]
        indent = "  " * level
        if node.type == NodeType.leaf:
            print(indent + str(node.edge_value) + " [Leaf node] class=" + node.val)
        else:
            cond = " <= " + str(node.split_cond) if ATTR[node.val].type == AttrType.num else " == ? "
            if node.type == NodeType.root:
                print("[Root node] '" + ATTR[node.val].label + "'" + cond)
            else:
                print(indent + str(node.edge_value) + " [Internal node] '" + ATTR[node.val].label + "'" + cond)
            # print tree for child notes recursively
            for n_id in node.children:
                self.print_model(n_id, level + 1)

    def build_model(self):
        self.__load_data()
        self.model = []  # holds the decision tree model, represented as a list of nodes
        # Get majority class
        #   Note: Counter returns a dictionary, most_common(x) returns a list with the x most common elements as
        #         (key, count) tuples; we need to take the first element of the list and the first element of the tuple
        # self.default_target = mean([x[IDX_TARGET] for x in self.data])
        target_list = []
        for x in self.data:
            target_list.append(x[IDX_TARGET])
        # self.default_target = [int(n) for n in target_list] TRENGER VI DEN??
        self.__id3(set(range(len(ATTR) - 1)), list(range(len(self.data))))

    def apply_model(self, record):
        node = self.model[0]
        while node.type != NodeType.leaf:
        # TODO based on the value of the record's attribute that is tested in `node`,# set `node` to one of its child nodes until a leaf node is reached
         return node.val
         
    def predict(self, records):
        predictions = []
        for record in records:
            pred_val = self.apply_model(self, record)
            # TODO append pred_val to predictions
        return predictions



   
A class RF representing the random forest classifier. It could represent with methods:

subsampling from the dataset
build a model with multiple decision trees
apply the model
    

    

    

    

    
    
    

In [111]:
class RF(object):
    def __init__(self):
        self.data = None  # training data set (loaded into memory)
        self.trees = None  # decision trees
    
    def __load_data(self):
        pass
        
    def __subsampling(self, train_set, sample_size_ratio):
        sample_number = round(len(self.data) * sample_size_ratio)
        # TODO: generate a subsample with replacement
    def build_model(self, train_set, split_conditions, sample_size_ratio, number_of_trees):
        for i in range(number_of_trees):
            sample = self.__subsampling(train_set, sample_size_ratio)
            tree = DT() # build a tree with sample data and split conditions
            self.trees.append(tree)

    def predict(self, test_set):
        rf_predictions = []
        for row in test_set:
            predictions = [tree.predict(row) for tree in self.trees]
            rf_predictions.append(np.mean(predictions))
        return rf_predictions

In [112]:
def main():
    dt = DT()
    dt.build_model()

if __name__ == "__main__":
    main()

attrs: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79}
records: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 15